### Testing with username

In [1]:
!pip install streamlit
!pip install pyngrok
!pip install praw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 7.7 MB/s eta 0:00:00


In [ ]:
%%writefile app04.py
import streamlit as st
import joblib
import pandas as pd
import praw

# Load the saved model and vectorizer
model = joblib.load('LRmodel.pkl')
vectorizer = joblib.load('LRvectorizer.pkl')

# Initialize Reddit API client
reddit = praw.Reddit(client_id='<REDDIT_CLIENT_ID>',
                     client_secret='<REDDIT_CLIENT_SECRET>',
                     user_agent='Mental Health')

# Function to fetch user posts
def fetch_user_posts(username):
    try:
        user = reddit.redditor(username)
        posts = [post.title + " " + post.selftext for post in user.submissions.new(limit=10)]
        return posts
    except Exception as e:
        st.write(f"Error fetching posts: {e}")
        return []

# Define the Streamlit app
def run_app():
    st.title('Mental Health Classifier with Username Test 04')

    # Reddit username input
    username = st.text_input("Enter Reddit username:")

    # Button to fetch posts and classify
    if st.button("Analyze"):
        if username.strip() == "":
            st.write("Please enter a Reddit username.")
        else:
            # Fetch posts
            posts = fetch_user_posts(username)
            if not posts:
                st.write("No posts found or unable to fetch posts for this user.")
            else:
                # Display some posts
                st.write("Some of the user's recent posts:")
                st.write(posts[:3])  # Display a few posts for review

                # Combine all posts into a single text
                combined_text = " ".join(posts)

                # Preprocess and vectorize the combined text
                input_vectorized = vectorizer.transform([combined_text])

                # Make prediction with probabilities
                prediction_proba = model.predict_proba(input_vectorized)

                # Get class labels and probabilities
                issue_labels = model.classes_
                proba_df = pd.DataFrame(prediction_proba, columns=issue_labels)
                proba_df = proba_df.T  # Transpose for readability
                proba_df.columns = ['Probability']

                # Get the mental health issue with the highest probability
                top_issue = proba_df['Probability'].idxmax()
                top_probability = proba_df['Probability'].max()

                # Output the result
                st.write(f"The most likely mental health concern is: {top_issue} with a probability of {top_probability:.2%}")

# Run the app
if __name__ == '__main__':
    run_app()


Writing app04.py


In [ ]:
# Import ngrok
from pyngrok import ngrok

# Set your authtoken
ngrok.set_auth_token("<NGROK_AUTH_TOKEN>") # Replace YOUR_AUTHTOKEN with your actual authtoken

# Kill any existing ngrok processes
ngrok.kill()

# Start Streamlit with nohup
!nohup streamlit run app04.py &

# Create a public URL with ngrok to access the app
public_url = ngrok.connect(addr='8501')
print(f"Public URL: {public_url}")

nohup: appending output to 'nohup.out'
Public URL: NgrokTunnel: "https://33ee-34-147-106-77.ngrok-free.app" -> "http://localhost:8501"


In [4]:
ngrok.kill()